# MLFA Assignment 2

Done by:
- Saumili Jana: 20ME10075
- Ushasi Das: 20EC10086
- Jaya Kishnani: 20EC30020
- Karthikeyan.R: 20EC30024

In [37]:
#importing modules
import math
import pandas as pd
import numpy as np

We import the data as panda's dataframes, split them into training and test dataset, and convert them into numpy array. As mentioned in the problem statement, we next split the data into 90% and 10% sets randomly, where 10% set is the test set.

In [38]:
#importing the dataset
all_data = pd.read_csv('sobar-72.csv')
print(all_data)

    behavior_sexualRisk  behavior_eating  behavior_personalHygine  \
0                    10               13                       12   
1                    10               11                       11   
2                    10               15                        3   
3                    10               11                       10   
4                     8               11                        7   
..                  ...              ...                      ...   
67                   10               14                       14   
68                   10               12                       15   
69                   10                8                       11   
70                    9               12                       13   
71                   10               14                       14   

    intention_aggregation  intention_commitment  attitude_consistency  \
0                       4                     7                     9   
1                      10

In [39]:
#reorganising the data into features and labels
all_arr = all_data.to_numpy() #converting to numpy arrays for ease of handling
print(all_arr)
X = all_arr[:,:-1]
Y = all_arr[:,-1]
print(X)
print(Y)

[[10 13 12 ... 11  8  1]
 [10 11 11 ...  4  4  1]
 [10 15  3 ...  3 15  1]
 ...
 [10  8 11 ... 10 10  0]
 [ 9 12 13 ... 13 15  0]
 [10 14 14 ... 15 15  0]]
[[10 13 12 ... 12 11  8]
 [10 11 11 ...  5  4  4]
 [10 15  3 ...  3  3 15]
 ...
 [10  8 11 ... 12 10 10]
 [ 9 12 13 ... 11 13 15]
 [10 14 14 ... 13 15 15]]
[1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]


In [40]:
#splitting data into training and test data
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size = 0.1) #10% set for testing
#print(X_train)
#print(y_test)
(row,col) =X_train.shape
#print(row,col)

#converting to float type - required for later stages
X_train =  np.asarray(X_train, dtype = np.float64, order ='C')
X_test =  np.asarray(X_test, dtype = np.float64, order ='C')
y_train =  np.asarray(y_train, dtype = np.float64, order ='C')
y_test =  np.asarray(y_test, dtype = np.float64, order ='C')

## Logistic regression using stochastic gradient ascent from scratch

We would be classifying the data using logistic regression and use stochastic gradient ascent to maximise the likelihood function. 

First we define functions for the logistic/sigmoid functions and to normalise/process the data

In [41]:
#Sigmoid/logistic function
def sigmoid(z):
    p = np.exp(z)/(1+np.exp(z))
    return(p)
#sigmoid(1)#debugging

In [42]:
#model parameters(theta_0,theta_1, theta_2,.......,theta_n) initialised to 0
parameters = np.zeros(20)
parameters

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0.])

In [43]:
#To normalise the data 
def normalise(array):
    k = 0
    while(k<col):
        ar = array[:,k]
        #print(ar)
        #print(np.max(ar))
        array[:,k] = ar/np.max(ar)
        k = k+1
    return(array)

X_train = normalise(X_train)
X_test= normalise(X_test)


Next we we obtain the expression for $z = \theta_0 +\theta_1x_1 + \theta_2x_2 + .......+\theta_nx_n $ where $\theta_0,\theta_1,....,\theta_n$ are the model parameters, and $x_1, x_2,..... ,x_n$ are the features for any given sample. Hence the probability of being in class1 is given by $P(y = 1|x) = \frac{e^{z}}{1+e^{z}}$.

In [44]:
#Linear regression function
def LR(feature,a):  #feature is an array containing all the features for an instance, 'a' is a vector containing model parameters
    z = 2
    count = 0
    while(count<col):
        z += a[count]*(feature[count])
        #print(z)
        count += 1
    p = sigmoid(z)
    #print(p)
    return(p)
#t = LR(X[18], parameters )   
#t

In [45]:
#Stochastic gradient ascent to maximise the likelihood function
itr = 1000 #no of iterations
r = 0.001 #learning rate


X_train_copy = X_train
while(itr>0):
    np.random.shuffle(X_train_copy) #shuffle the set
    j = col
    while(j>0):
        i = 0
        for x_i in X_train_copy:
            parameters[col-j] = parameters[col-j]+ r*((y_train[i]-LR(x_i,parameters))*x_i[col-j])
            #print(i,col-j)
            i = i+1
        j = j-1
    #print(itr, parameters)
    itr = itr-1
parameters

array([-0.65157309, -0.51370713, -0.19354134, -0.19566284, -0.42783572,
       -0.36877652, -0.51495517, -0.06118446, -0.06019931, -0.08908024,
        0.03373948, -0.26277548,  0.03712069, -0.04313761, -0.08574156,
       -0.17263902,  0.10580238,  0.02195223,  0.02947995,  0.        ])

In [46]:
#classify function - returns 0 or 1 according to its predicted class
def classify(sample, a):
    p = LR(sample,a)
    if p>0.5:
        return(1)
    else:
        return(0)


In [47]:
#predicting labels
y_pred = [] #predicted labels of the test dataset
for b in X_test:
    label = classify(b,parameters)
    y_pred.append(label)
print(y_pred)
#y_test

[0, 0, 0, 0, 0, 0, 0, 0]


Next we find the accuracy of our prediction

In [48]:
#Accuracy function 
def accuracy(pred , y_test):
    count = 0
    for i in range(len(pred)):
        if pred[i] == y_test[i]:
            count +=1
            
    return (count/len(pred))*100


In [49]:
#finding the accuracy
acc = accuracy(y_pred,y_test)
print("Accuracy = ",acc, " %")

Accuracy =  75.0  %


## Using sklearn libraries

In [50]:
#To filter out un-necessary warnings
import warnings
warnings.filterwarnings( "ignore" )

In [51]:
#Model Training on train set
from sklearn.linear_model import LogisticRegression
model = LogisticRegression()
model.fit(X_train, y_train)

LogisticRegression()

In [52]:
#Prediction on test set
Y_predict = model.predict(X_test)
Y_predict

array([0., 0., 0., 0., 0., 0., 0., 0.])

In [53]:
#Accuracy
score = model.score(X_test, y_test)
print("Accuracy using sklearn= ",score*100, " %")

Accuracy using sklearn=  75.0  %


### Writing the final results in a file

In [54]:
file = open('results.txt','w')
file.write("\t\t\tFINAL RESULTS\n\t\t       MLFA Assignment-2\n--------------------------------------------------------------\n--------------------------------------------------------------\n")
str1 = "Accuracy of the prediction using code written from scratch is " + str(acc) + " % "
file.write(str1)
str2 = "\nAccuracy of the prediction using sklearn model is " + str(score*100) + " % "
file.write(str2)
file.write("\n--------------------------------------------------------------\n--------------------------------------------------------------\n")
file.close()